# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [1]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots
using LaTeXStrings

In [2]:
include("../../../src/decompositions/tensors/naive_low_multilinear_rank_approximation.jl")
include("../../../src/decompositions/tensors/curvature_corrected_low_multilinear_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")

exact_loss (generic function with 2 methods)

### Load data and construct manifold ###

In [3]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("../../../data/nifti_dt_nonlinear.nii.gz") * 1e9;
size(ni)

(112, 112, 50, 1, 6)

In [4]:
 d1, d2, d3, _, d = size(ni)

# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [ # data ordered as [xx, yx, yy, zx, zy, zz]
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];
    
# pick a 2D slice 
data = predata[34:81,32:79,15];
data = data[1:10,1:10]

# construct data manifold
n = size(data)
M = SymmetricPositiveDefinite(3)
d = manifold_dimension(M) # -> different than in 1D

6

In [25]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

### Construct low rank approximation ###

In [5]:
unrolled_data = []
for mat in data
    push!(unrolled_data,mat)
end
q = mean(M, unrolled_data)
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^n

In [27]:
# Time loss evaluation
# curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
# Time gradient evaluation

In [6]:
# TODO find a good step size...
_, __ = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [2,2]; stepsize=1/1000, max_iter=2, change_tol=1e-4); 

Initial  F(x): 0.27325725719 | 
# 1     change: 0.283441623 |  F(x): 0.22836737992 | 
# 2     change: 0.208307854 |  F(x): 0.20397576038 | 
The algorithm reached its maximal number of iterations (2).


In [29]:
rank_range = [1, 2]
# rank_range = [1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 48]
max_iter = 5

nR_q = []
nU = []
ccR_q = []
ccU = []
eR_q = []
eU = []
for rank in rank_range
    println("#$(rank) | computing naive low-rank approximation")
    nRr_q, nUr = naive_low_multilinear_rank_approximation(M, q, data, [rank,rank]); 
    push!(nR_q, nRr_q)
    push!(nU, nUr)
    println("#$(rank) | computing curvature corrected low-rank approximation")
    ccRr_q, ccUr = curvature_corrected_low_multilinear_rank_approximation(M, q, data, [rank,rank]; stepsize=1/1000, max_iter=max_iter, change_tol=1e-4); 
    push!(ccR_q, ccRr_q)
    push!(ccU, ccUr)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
Initial  F(x): 0.57435644879 | 
# 1     change: 0.507565740 |  F(x): 0.38829214031 | 
# 2     change: 0.332052537 |  F(x): 0.30843379000 | 
# 3     change: 0.217295518 |  F(x): 0.27404702564 | 
# 4     change: 0.142280955 |  F(x): 0.25914720129 | 
# 5     change: 0.093267900 |  F(x): 0.25261396276 | 
The algorithm reached its maximal number of iterations (5).
#2 | computing naive low-rank approximation
#2 | computing curvature corrected low-rank approximation
Initial  F(x): 0.27325725719 | 
# 1     change: 0.283441623 |  F(x): 0.22836737992 | 
# 2     change: 0.208307854 |  F(x): 0.20397576038 | 
# 3     change: 0.153130398 |  F(x): 0.19067394036 | 
# 4     change: 0.112613654 |  F(x): 0.18338022997 | 
# 5     change: 0.082867843 |  F(x): 0.17934845412 | 
The algorithm reached its maximal number of iterations (5).


In [30]:
num_ranks = length(rank_range)
ref_distance = sum(distance.(Ref(M), Ref(q), data).^2)

naive_tangent_distances_r = zeros(num_ranks)
predicted_naive_distances_r= zeros(num_ranks)
true_naive_distances_r= zeros(num_ranks)

curvature_corrected_tangent_distances_r = zeros(num_ranks)
predicted_curvature_corrected_distances_r = zeros(num_ranks)
true_curvature_corrected_distances_r = zeros(num_ranks)

for (idx,rank) in enumerate(rank_range)
    naive_log_q_data_r = Symmetric.([sum([nR_q[idx][i,j] * nU[idx][1][k,i] * nU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    curvature_corrected_log_q_data_r = Symmetric.([sum([ccR_q[idx][i,j] * ccU[idx][1][k,i] * ccU[idx][2][l,j] for i in 1:rank, j in 1:rank]) for k=1:n[1], l=1:n[2]])
    # expoentiate back
    naive_data_r = exp.(Ref(M), Ref(q), naive_log_q_data_r)
    curvature_corrected_data_r = exp.(Ref(M), Ref(q), curvature_corrected_log_q_data_r)

    # compute relative tangent space error
    naive_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - naive_log_q_data_r).^2) / ref_distance
    curvature_corrected_tangent_distances_r[idx] = sum(norm.(Ref(M), Ref(q),  log_q_data - curvature_corrected_log_q_data_r).^2) / ref_distance

    # compute relative manifold error
    predicted_naive_distances_r[idx] = curvature_corrected_loss(M, q, data, naive_log_q_data_r)
    true_naive_distances_r[idx] = exact_loss(M, q, data, naive_log_q_data_r)
    predicted_curvature_corrected_distances_r[idx] = curvature_corrected_loss(M, q, data, curvature_corrected_log_q_data_r)
    true_curvature_corrected_distances_r[idx] = exact_loss(M, q, data, curvature_corrected_log_q_data_r)
end

In [31]:
plot(rank_range, [naive_tangent_distances_r, true_naive_distances_r, true_curvature_corrected_distances_r], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(0,1), xlims=(1,max(rank_range...)),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/Camino2D_errors_by_rank.png")
plot(rank_range, [naive_tangent_distances_r .+ 1e-16, true_naive_distances_r .+ 1e-16, true_curvature_corrected_distances_r .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected"], ylims=(1e-16,1), xlims=(1,max(rank_range...)), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logerrors_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logerrors_by_rank.png"

In [32]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
plot(rank_range, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(rank_range...)),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/Camino2D_discrepancy_by_rank.png")
plot(rank_range, [predicted_naive_distances_r ./ true_naive_distances_r, predicted_curvature_corrected_distances_r ./ true_curvature_corrected_distances_r], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,max(rank_range...)), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/Camino2D_logdiscrepancy_by_rank.png")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/2D/P3/results/Camino2D_logdiscrepancy_by_rank.png"